# Step 1 - Importing all libraries used throughout the project

In [ ]:
import functions_vanguard as vd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind

# Step 2 - Importing data frames, analyzing and cleaning

## 2.1.1 - Importing Clients Dataframe (df_final_demo)

In [ ]:
df_final_demo = vd.import_dataframe()
df_final_demo
   

## 2.1.2 Exploring the Dataframe's columns

*Display basic information about the dataset*, 

*Displaying information about **clnt_tenure_yr** (client tenure in years), as well as plotting a histogram to visualize the column's values*

*Displaying information about **clnt_age** (client age), as well as plotting a histogram to visualize the column's values*

*Displaying information about **gendr** (gender), as well as creating a pie chart to visualize the column's values*

*Displaying iformation about **num_accounts** (number of accounts), as well as creating a pie chart and histogram to visualize the column's values*

*Displaying iformation about **bal** (balance)*

*Displaying iformation about **logons_6_mnth** (log on in the past 6 months), as well as creating a histogram to visualize the column's values*

In [ ]:
vd.analyze_dataframe(df_final_demo)

## 2.1.3 - Analyzing Client Demographics

First, we will create a new column to visualize the total tenure in months.

Second, we will categorize the the clients based on their total tenure (in months). They can either be **New** or **Long Standing**

Third, it is important to know whether the clients are **young** or **old**. 

In [ ]:
df_final_demo = vd.analyze_client_demographics(df_final_demo)
df_final_demo

## 2.1.3 - Identifying Primary Clients

In [ ]:
vd.identify_primary_clients(df_final_demo)

## 2.1.4 Cleaning df_final_demo 

First, we will take a look at which columns are completly empty (except for client_id, whic does not have any null values).

Secondly, we will remove all these rows.

Finally, we will take a look a the types of each column and make adjustments if needed.

In [ ]:
df_final_demo=vd.clean_dataframe(df_final_demo)
df_final_demo

## 2.2.1 Importing DataFrame Part 1 which provides information about the visits to Vanguard

In [ ]:
df_pt1 = vd.import_and_check_dataframe_part1()
df_pt1

There aren't any null values, so we do not have to clean anything.

## 2.3.1 - Importing DataFrame Part 2 

In [ ]:
df_pt2 = vd.import_and_check_dataframe_part2()
display(df_pt2)

## 2.3.2 - Merging the Twin Dataframes

DataFrame Part 2 is an extension to DataFrame Part 1; therefore, they should be merged.

In [ ]:
df = vd.merge_dataframes(df_pt1, df_pt2)
df

## 2.4.1 - Importing Data Frame about the different groups of clients (Test/Control)

We want to determine the **size** of each of the groups. 

Determining the amount of **null values** in the Dataframe

In [ ]:
df_final_experiment = vd.import_and_analyze_experiment_clients()
df_final_experiment

In [ ]:
new_df = pd.merge(df_final_demo, df, how="left", on = "client_id")

variation_df = pd.merge(new_df, df_final_experiment_clients, on = "client_id", how="inner") # We are using an inner join bc we do not care about client's that did not go thru the process
variation_df.reset_index()
variation_df

In [ ]:
control_df = variation_df[variation_df["Variation"] == "Control"]
test_df = variation_df[variation_df["Variation"] == "Test"]

# Step 3 - Determining KPI's

Analyze Time Spent on Each Step

In [ ]:
# Convert date_time to datetime format
df['date_time'] = pd.to_datetime(df['date_time'])

In [ ]:
# Sort by visit_id and date_time
df = df.sort_values(by=['visit_id', 'date_time'])

In [ ]:
# Calculate time spent on each step
df['time_spent'] = df.groupby('visit_id')['date_time'].diff().dt.total_seconds()

In [ ]:
# Fill NaN values in time_spent with 0 for the first step
df['time_spent'] = df['time_spent'].fillna(0)

In [ ]:
# Calculate average time spent on each step
time_spent_summary = df.groupby('process_step')['time_spent'].mean().reset_index()
print("Average Time Spent on Each Step:")
print(time_spent_summary)


In [ ]:
# Plotting the average time spent on each step
plt.figure(figsize=(10, 6))
sns.barplot(x='process_step', y='time_spent', data=time_spent_summary, palette='viridis')

# Adding titles and labels
plt.title('Average Time Spent on Each Step')
plt.xlabel('Process Step')
plt.ylabel('Average Time Spent (seconds)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
from scipy.stats import ttest_ind

# Calculate session duration for each visit_id
session_durations = variation_df.groupby('visit_id')['time_spent'].sum().reset_index()

# Merge with the original data to get the Variation labels
df_merged = session_durations.merge(variation_df[['visit_id', 'Variation']].drop_duplicates(), on='visit_id')

# Separate the data into control and test groups
control_group = df_merged[df_merged['Variation'] == 'Control']['time_spent']
test_group = df_merged[df_merged['Variation'] == 'Test']['time_spent']

# Perform t-test
t_stat, p_value = ttest_ind(test_group, control_group, alternative='greater')

# Output the results
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. Clients using the new UI have significantly longer session durations.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in session durations.")

In [ ]:
process_step_counts = df['process_step'].value_counts()
process_step_counts

In [ ]:
# Count the occurrences of each process step
process_step_counts = df['process_step'].value_counts()

# Plot a bar chart
plt.figure(figsize=(10, 6))
process_step_counts.plot(kind='bar', color='skyblue')
plt.title('Distribution of Process Steps')
plt.xlabel('Process Step')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y')

# Display the plot
plt.show()

# KPI's

In [ ]:
variation_df.dropna(subset="Variation", inplace = True)

In [ ]:
# Total number of sessions for each group
total_sessions = variation_df.groupby('Variation')['visit_id'].nunique().reset_index(name='total_sessions')

# Filter the data to only include rows where process_step is "confirm"
confirm_steps = variation_df[variation_df['process_step'] == 'confirm']

# Count the number of sessions that reached the "confirm" step for each group
confirm_sessions = confirm_steps.groupby('Variation')['visit_id'].nunique().reset_index(name='confirm_sessions')

#Merge total sessions with confirm sessions
completion_data = pd.merge(total_sessions, confirm_sessions, on='Variation')

# Calculate the completion rate
completion_data['completion_rate'] = (completion_data['confirm_sessions'] / completion_data['total_sessions']) * 100

print(completion_data)

In [ ]:
#create a bar chart
plt.figure(figsize=(5, 3))
sns.barplot(x='Variation', y='completion_rate', data=completion_data, palette='Set2')
plt.title('Completion Rate by Variation')
plt.xlabel('Variation')
plt.ylabel('Completion Rate (%)')
plt.ylim(0, 100)
plt.show()

In [ ]:
# Calculate the observed increase in completion rate
completion_rate_control = completion_data.loc[completion_data['Variation'] == 'Control', 'completion_rate'].values[0]
completion_rate_test = completion_data.loc[completion_data['Variation'] == 'Test', 'completion_rate'].values[0]
observed_increase = completion_rate_test - completion_rate_control
print(f"Observed Increase in Completion Rate: {observed_increase:.2f}%")

import numpy as np
from scipy.stats import chi2_contingency

confirm_sessions_control = 16046
total_sessions_control = 32189
confirm_sessions_test = 21731
total_sessions_test = 37136

# Create the contingency table
contingency_table = np.array([
    [confirm_sessions_control, total_sessions_control - confirm_sessions_control],
    [confirm_sessions_test, total_sessions_test - confirm_sessions_test]
])

# Perform the chi-squared test
chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)

# Print the test statistic and p-value
print(f"Chi-Squared Statistic: {chi2_stat:.2f}")
print(f"P-Value: {p_value:.4f}")

threshold = 5.0  # 5% threshold
if observed_increase >= threshold and p_value < 0.05:
    print("The observed increase in completion rate meets or exceeds the 5% threshold and is statistically significant.")
else:
    print("The observed increase in completion rate does not meet the 5% threshold or is not statistically significant.")


In [ ]:
#Amoutn of people per group.
variation_counts = variation_df["Variation"].value_counts()
variation_counts.plot(kind="bar", color = "lightskyblue" )
plt.xlabel("Group")
plt.ylabel("Frequency")

In [ ]:
print(control_df.client_id.count(),test_df.client_id.count())

In [ ]:
# # Function to calculate time spent on each step
# def calculate_step_duration(df):
#     # Ensure 'date_time' is sorted within each user session
#     df = df.sort_values(by=['client_id', 'visitor_id', 'date_time'])
    
#     # Calculate the time spent on each step by finding the difference between consecutive steps
#     df['next_step_time'] = df.groupby(['client_id', 'visitor_id'])['date_time'].shift(-1)
#     df['time_spent'] = (df['next_step_time'] - df['date_time']).dt.total_seconds()
    
#     # Drop rows where 'time_spent' is NaN (e.g., the last step in each session)
#     df = df.dropna(subset=['time_spent'])
    
#     # Calculate the average duration spent on each step
#     avg_duration_per_step = df.groupby('process_step')['time_spent'].mean().reset_index()
#     avg_duration_per_step.columns = ['process_step', 'avg_duration_seconds']
    
#     return avg_duration_per_step

# # Apply the function
# average_duration_per_step = calculate_step_duration(variation_df)

# # Print the results
# print("Average Duration Spent on Each Step (in seconds):")
# print(average_duration_per_step)

In [ ]:
# plt.figure(figsize=(10, 6))
# sns.barplot(x='process_step', y='avg_duration_seconds', data=average_duration_per_step, palette='viridis')
# plt.title('Average Duration Spent on Each Step (in seconds)')
# plt.xlabel('Process Step')
# plt.ylabel('Average Duration (seconds)')
# plt.xticks(rotation=45)
# plt.show()

In [ ]:


# # General function to calculate time spent on each step
# def calculate_step_duration(df):
#     # Ensure 'date_time' is sorted within each user session
#     df = df.sort_values(by=['client_id', 'visitor_id', 'date_time'])
    
#     # Calculate the time spent on each step by finding the difference between consecutive steps
#     df['next_step_time'] = df.groupby(['client_id', 'visitor_id'])['date_time'].shift(-1)
#     df['time_spent'] = (df['next_step_time'] - df['date_time']).dt.total_seconds()
    
#     # Drop rows where 'time_spent' is NaN (e.g., the last step in each session)
#     df = df.dropna(subset=['time_spent'])
    
#     # Calculate the average duration spent on each step
#     avg_duration_per_step = df.groupby('process_step')['time_spent'].mean().reset_index()
#     avg_duration_per_step.columns = ['process_step', 'avg_duration_seconds']
    
#     return avg_duration_per_step

# # Apply the function to control and test dataframes
# average_duration_per_step_control = calculate_step_duration(control_df)
# average_duration_per_step_test = calculate_step_duration(test_df)

# # Print the results
# print("Average Duration Spent on Each Step (Control Group) (in seconds):")
# print(average_duration_per_step_control)

# print("Average Duration Spent on Each Step (Test Group) (in seconds):")
# print(average_duration_per_step_test)



In [ ]:

# Assuming 'variation_df' is your DataFrame and it includes 'date_time' and 'process_step' columns
control_df = variation_df[variation_df["Variation"] == "Control"]
test_df = variation_df[variation_df["Variation"] == "Test"]

# General function to calculate time spent on each step
def calculate_step_duration(df):
    # Ensure 'date_time' is sorted within each user session
    df = df.sort_values(by=['client_id', 'visitor_id', 'date_time'])
    
    # Calculate the time spent on each step by finding the difference between consecutive steps
    df['next_step_time'] = df.groupby(['client_id', 'visitor_id'])['date_time'].shift(-1)
    df['time_spent'] = (df['next_step_time'] - df['date_time']).dt.total_seconds()
    
    # Drop rows where 'time_spent' is NaN (e.g., the last step in each session)
    df = df.dropna(subset=['time_spent'])
    
    # Calculate the average duration spent on each step
    avg_duration_per_step = df.groupby('process_step')['time_spent'].mean().reset_index()
    avg_duration_per_step.columns = ['process_step', 'avg_duration_seconds']
    
    return avg_duration_per_step

# Apply the function to control and test dataframes
average_duration_per_step_control = calculate_step_duration(control_df)
average_duration_per_step_test = calculate_step_duration(test_df)

# Add a column to distinguish between control and test group
average_duration_per_step_control['Group'] = 'Control'
average_duration_per_step_test['Group'] = 'Test'


In [ ]:
# Combine both dataframes
combined_df = pd.concat([average_duration_per_step_control, average_duration_per_step_test])

# Plotting
plt.figure(figsize=(12, 6))

# Create separate palettes for control and test groups
palette = {'Control': 'skyblue', 'Test': 'coral'}

# Create a bar plot using Seaborn
sns.barplot(x='process_step', y='avg_duration_seconds', hue='Group', data=combined_df, palette=palette)

# Set plot title and labels
plt.title('Average Duration Spent on Each Step by Group (in seconds)')
plt.xlabel('Process Step')
plt.ylabel('Average Duration (seconds)')

# Show plot
plt.show()

In [ ]:
# Assuming you have DataFrames `test_df` and `control_df` with a column 'process_step'

# Count the occurrences of each unique value in the 'process_step' column for both DataFrames
process_step_counts_test = test_df['process_step'].value_counts().rename('Test')
process_step_counts_control = control_df['process_step'].value_counts().rename('Control')

# Normalize the counts by the total number of entries in each group
total_test = len(test_df)
total_control = len(control_df)

normalized_test_counts = process_step_counts_test / total_test
normalized_control_counts = process_step_counts_control / total_control

# Combine the normalized counts into a single DataFrame
combined_counts = pd.concat([normalized_test_counts, normalized_control_counts], axis=1).fillna(0)

# Plot a bar chart
plt.figure(figsize=(12, 8))
combined_counts.plot(kind='bar', color=['red', 'skyblue'], width=0.8)
plt.title('Proportional Distribution of Process Steps (Test vs Control Group)')
plt.xlabel('Process Step')
plt.ylabel('Proportion')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')

# Display the plot
plt.show()

**Error Rates**

#Sort the data by client_id, visit_id, and date_time to trace the users’ navigation sequence accurately.

In [ ]:
# Sort data by client_id, visit_id, and date_time
variation_df = variation_df.sort_values(by=['client_id', 'visitor_id', 'date_time'])


#Convert time_period to a numeric type: We need to convert time_period to a numeric type if it's not already one, and ensure prev_step is also numeric.

In [ ]:
# Ensure 'time_period' is numeric
# variation_df['time_period'] = pd.to_numeric(variation_df['time_period'], errors='coerce')

Determine the frequency of backward navigations for each user in both control and test groups.
Compare the error rates between the two groups to assess if the new UI design reduces or increases errors.

Creating prev_step:

Operation: df.groupby(['client_id', 'visitor_id'])['time_period'].shift(1)
Purpose: This groups the data by client_id and visitor_id, then shifts the time_period column by one position to get the previous step for each user session.
Result: A new column prev_step that shows the previous time_period for each step.

Calculating is_backward:

Operation: df.apply(lambda row: row['time_period'] < row['prev_step'] if pd.notnull(row['prev_step']) else False, axis=1)
Purpose: For each row, compare the current time_period with prev_step. If prev_step is not NaN, check if the current time_period is less than prev_step, indicating a backward navigation. If prev_step is NaN, set is_backward to False.

Result: A new column is_backward that is True if a backward navigation occurred, otherwise False.

In [ ]:
#variation_df[['client_id', 'visitor_id', 'visit_id', 'date_time', 'prev_step','Variation', 'is_backward','process_step','step_index','prev_step_index', 'is_back_track']].head(20)

In [ ]:
#variation_df['is_back_track'].value_counts()

In [ ]:
# Create the new column with the mapped values
mapping = { 'start': 0, 'step_1': 1, 'step_2': 2, 'step_3': 3, 'confirm': 4 } 
variation_df['step_index'] = variation_df['process_step'].map(mapping)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
variation_df.sort_values(by=['visit_id', 'date_time'])

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
variation_df['prev_step_index'] = variation_df.groupby('visit_id')['step_index'].shift(1)


In [ ]:
# Function to detect backward navigation
variation_df['is_back_track'] = variation_df['prev_step_index'] > variation_df['step_index'] 

Calculated error rates by grouping the data by the variation column and computing the mean of the is_backward column.

In [ ]:
# Calculate error rates
error_rates = variation_df.groupby('Variation')['is_back_track'].mean()

print("Error Rates:")
print(error_rates)

Perform statistical tests (e.g., chi-square test) to determine if the difference in error rates between the control and test groups is statistically significant.

In [ ]:
# Statistical analysis (e.g., chi-square test)
# Create a contingency table
contingency_table = pd.crosstab(variation_df['Variation'], variation_df['is_back_track'])

chi2, p, dof, ex = chi2_contingency(contingency_table)

print(f"Chi-Square Test:\nChi2: {chi2}\np-value: {p}")

In [ ]:
variation_df

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Calculate session duration for each visit_id
session_durations = variation_df.groupby('visit_id')['time_spent'].sum().reset_index()

# Merge with the original data to get the Variation labels
df_merged = session_durations.merge(variation_df[['visit_id', 'Variation']].drop_duplicates(), on='visit_id')

# Separate the data into control and test groups
control_group = df_merged[df_merged['Variation'] == 'Control']['time_spent']
test_group = df_merged[df_merged['Variation'] == 'Test']['time_spent']

# Perform t-test
t_stat, p_value = ttest_ind(test_group, control_group, alternative='greater')

# Output the results
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. Clients using the new UI have significantly longer session durations.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference in session durations.")



In [ ]:
variation_df.Variation.isna().sum()

In [ ]:
# Visualization
df_merged['group'] = np.where(df_merged['Variation'] == 'Test', 'Test Group', 'Control Group')

plt.figure(figsize=(12, 6))
sns.boxplot(x='group', y='time_spent', data=df_merged, palette='Set2')

plt.title('Session Durations: Control vs. Test Group')
plt.xlabel('Group')
plt.ylabel('Session Duration (seconds)')
plt.show()

In [ ]:
# Assuming df_merged is already defined and has the necessary data
df_merged['group'] = np.where(df_merged['Variation'] == 'Test', 'Test Group', 'Control Group')

plt.figure(figsize=(12, 6))

# Scatter plot
sns.scatterplot(x='group', y='time_spent', data=df_merged, hue='group', palette='Set2')

plt.title('Session Durations: Control vs. Test Group')
plt.xlabel('Group')
plt.ylabel('Session Duration (seconds)')
plt.legend(title='Group')
plt.show()

Analyze error rates by different segments (e.g., client tenure, age, gender, number of accounts, balance) to identify if specific groups are more prone to errors.
This can help tailor future improvements or targeted interventions for specific client segments.

In [ ]:
# Segmentation analysis (example by client tenure)
segmentation_analysis = variation_df.groupby(['Variation', 'clnt_tenure_yr'])['is_back_track'].mean().unstack()
print("Segmentation Analysis by Client Tenure:")
print(segmentation_analysis)

This approach will help you understand how the new UI affects user behavior, specifically focusing on error rates indicated by backward navigation. The analysis will also provide insights into which segments of users are most affected, enabling more targeted improvements.

In [ ]:
# Reset index for plotting
segmentation_analysis_reset = segmentation_analysis.reset_index()

# Melt the DataFrame to long format for easier plotting with seaborn
segmentation_analysis_melted = segmentation_analysis_reset.melt(id_vars='Variation', var_name='Client Tenure (years)', value_name='Backtrack Rate')

# Visualization
plt.figure(figsize=(16, 7))
sns.barplot(x='Client Tenure (years)', y='Backtrack Rate', hue='Variation', data=segmentation_analysis_melted, palette='viridis')

# Adding titles and labels
plt.title('Backtrack Rate by Client Tenure and Variation')
plt.xlabel('Client Tenure (years)')
plt.ylabel('Mean Backtrack Rate')
plt.legend(title='Variation')
plt.show()

In [ ]:
# Visualization 
sns.barplot(x='Variation', y='is_back_track', data=variation_df)
plt.title('Error Rates by Variation')
plt.xlabel('Variation')
plt.ylabel('Error Rate')
plt.show()

Compute the Time Spent on Each Step: Calculate the difference between the timestamps of consecutive steps for each user session.
Aggregate the Time Spent by Step: Average these durations for each step across all user sessions.

Create next_step_time: Use shift(-1) to get the timestamp of the next step within each user session.
Calculate time_spent: Find the difference between next_step_time and date_time, then convert this to seconds.

Drop Rows with NaN time_spent:
Remove rows where time_spent is NaN, which would occur for the last step of each session since there is no next step to compare.
Calculate Average Duration:

Group by process_step and compute the mean of time_spent for each step.
Reset the index and rename columns for clarity.

In [ ]:
variation_df.to_csv('variation.csv', index=False)

THE KPIS ARE : Completion rate, error rate, time spent.